In [1]:
#!pip install dpkt

In [106]:
import os
import dpkt
from tqdm.auto import tqdm
import datetime
import numpy as np

In [107]:
pcap_path="."
pcap_file="Velo.01.pcap"
full_path = os.path.join(pcap_path, pcap_file)

In [108]:
fpcap = open(full_path, 'rb')
lidar_reader = dpkt.pcap.Reader(fpcap)

# check pcap length
max_pcap_counter = 0
# itearte through each data packet and timestamps
for _, _ in enumerate(lidar_reader):
    max_pcap_counter += 1
fpcap.close()
print(f'max_pcap_counter={max_pcap_counter}')

max_pcap_counter=182036


In [109]:
fpcap = open(full_path, 'rb')
lidar_reader = dpkt.pcap.Reader(fpcap)

index_from = 0
index_to = max_pcap_counter
#index_to = 20
anal_pcap_len = index_to - index_from - 1
pbar = tqdm(total=anal_pcap_len)
data_pcap = []
for idx, (ts, buf) in enumerate(lidar_reader):
    
    if idx <= index_from:
        continue
    if index_to <= idx:
        break
        
    strDatetime = datetime.datetime.utcfromtimestamp(ts)
    eth = dpkt.ethernet.Ethernet(buf)
    #print(f'eth.data.data.sport={eth.data.data.sport}')
    if eth.data.data.sport == 2368: # self.params['data-port'] = 2368
        data = eth.data.data.data
        data_pcap.append(data)

    #
    #process_data_frame(data, ts, idx)
    #print(f'strDatetime={strDatetime}')

    pbar.update(1)

  0%|          | 0/182035 [00:00<?, ?it/s]

In [110]:
def read_uint32(data, idx):
    return data[idx] + data[idx+1]*256 + data[idx+2]*256*256 + data[idx+3]*256*256*256

def read_firing_data(data):
    block_id = data[0] + data[1]*256
    # 0xeeff is upper block
    assert block_id == 0xeeff

    azimuth = (data[2] + data[3] * 256) / 100 # 왜 100으로? 0.01도를 1로 세팅한 것으로 보임. 

    firings = data[4:].reshape(32, 3)
    distances = firings[:, 0] + firings[:, 1] * 256
    distances = distances/100.
    intensities = firings[:, 2]
    return distances, intensities, azimuth # 두개의 데이터가 스택되어 있다는 점 중요. 

def calc_precise_azimuth_2(azimuth):
    org_azi = azimuth.copy()

    precision_azimuth = []
    # iterate through each block
    for n in range(12): # n=0..11
        azimuth = org_azi.copy()
        # Determine the azimuth Gap between data blocks
        if n < 11:
            # First, adjust for an Azimuth rollover from 359.99° to 0°
            if azimuth[n + 1] < azimuth[n]:
                azimuth[n + 1] += 360.
            azimuth_gap = azimuth[n + 1] - azimuth[n]
        else:
            # Last block.  Simply use last iteration `azimuth_gap`.
            pass

        # iterate through each firing
        for k in range(32):
            # Determine if you’re in the first or second firing sequence of the data block
            if k < 16:
                # Interpolate
                precise_azimuth = azimuth[n] + (azimuth_gap * 2.304 * k) / (2 * 55.296)
            else:
                # Interpolate, but 55.296 us has passed.
                precise_azimuth = azimuth[n] + (azimuth_gap * (55.296 + 2.304*(k - 16))) / (2 * 55.296)
            if precise_azimuth >= 360.:
                precise_azimuth -= 360.
                #print(precise_azimuth)
            precision_azimuth.append(precise_azimuth)
    precision_azimuth = np.array(precision_azimuth)             
    #print(f'*shape of precision_azimuth={precision_azimuth.shape}')
    return precision_azimuth

def calc_cart_coord(distances, azimuth): # X, Y, Z 생성에 대하여 완전히 이해함. 
    # convert distances to meters
    distances = distances * FACTOR_MM2CM * FACTOR_CM2M

    # convert deg to rad
    longitudes = np.tile(npa_omega * np.pi / 180., 2)
    latitudes = azimuth * np.pi / 180.

    hypotenuses = distances * np.cos(longitudes)

    X = hypotenuses * np.sin(latitudes)
    Y = hypotenuses * np.cos(latitudes)
    Z = distances * np.sin(longitudes)

    # Apply vertical correction for laser positions
    Z += 1e-3 * np.tile(npa_vcorr_mm, 2)

    return X, Y, Z

def calc_timing_offsets(dual_mode=False):
    timing_offsets = [[0.0 for x in range(12)] for y in range(32)]  # Init matrix

    # constants
    full_firing_cycle = 55.296  # μs
    single_firing = 2.304  # μs
    # compute timing offsets
    for x in range(12):
        for y in range(32):
            if dual_mode:
                dataBlockIndex = (x - (x % 2)) + int((y / 16))
            else:
                dataBlockIndex = (x * 2) + int((y / 16))
            dataPointIndex = y % 16
            timing_offsets[y][x] = \
                (full_firing_cycle * dataBlockIndex) + (single_firing * dataPointIndex)
    return np.array(timing_offsets).T

def attach_2d_list(tlist2d, x, newRow):
    #print(f'len(t)={len(tlist2d)}')
    if newRow or (len(tlist2d)==0):
        tlist2d = tlist2d+[x]
        #print(f'x={x} for hstack. Finally t={tlist2d}') 
    else:

        tlist2d[-1]=tlist2d[-1]+x
        #print(f'x={x} for append at last row. Finally t={tlist2d}')
    return tlist2d

def attach_range_image_list(t2DImageList, vertical32, newRow):
    #print(f'len(t)={len(tlist2d)}')
    if newRow or (len(t2DImageList)==0):
        t2DImageList = t2DImageList+[[[t] for t in vertical32[0:16][change_vertical_position]]]
        t2DImageList[-1] = [t1+[t2] for t1, t2 in zip(t2DImageList[-1], vertical32[16:32][change_vertical_position])]
        #print(f'newRow={newRow}, t2DImageList={t2DImageList} for a new image') 
    else:
        t2DImageList[-1] = [t1+[t2] for t1, t2 in zip(t2DImageList[-1], vertical32[0:16][change_vertical_position])]
        t2DImageList[-1] = [t1+[t2] for t1, t2 in zip(t2DImageList[-1], vertical32[16:32][change_vertical_position])]
        #print(f'newRow={newRow}, t2DImageList={t2DImageList} for the existing image') 
    return t2DImageList

In [111]:
def write_rimg(path, distance):
    print('distance.shape: ', distance.shape)
    points = int(len(distance)/16)
    distance = distance.astype(np.float16).reshape(1, -1)#(16, points)
    print('distance.shape: ', distance.shape, ' points: ', points)
    size_array = np.array([ 16,0, points, 0])
    print('Writing array:', size_array)
    fp = open(path, 'wb')
    fp.write(size_array.tostring())
    fp.write(distance.tostring())
    fp.close()
    print('Writing Completed.')

In [112]:
FACTOR_MM2CM = 0.2 # 왜? 0.1이 답일텐데... <= 비행거리 time-of-flight가 왕복이기 때문에 2배를 곱한 것. 
FACTOR_CM2M = 0.01
npa_omega = np.array([-15, 1, -13, 3, -11, 5, -9, 7, -7, 9, -5, 11, -3, 13, -1, 15])
npa_vcorr_mm = np.array([11.2, -0.7, 9.7, -2.2, 8.1, -3.7, 6.6, -5.1, 5.1, -6.6, 3.7, -8.1, 2.2, -9.7, 0.7, -11.2])
npa_timing_offsets = calc_timing_offsets()

# 각도 순서에 따른 재배치
change_vertical_position = [index for index, value in sorted(enumerate(npa_omega), key=lambda x: x[1])]

'''
data = [-15, 1, -13, 3, -11, 5, -9, 7, -7, 9, -5, 11, -3, 13, -1, 15]
# 원소에 인덱스를 붙입니다.
indexed_data = list(enumerate(data))
# 값을 기준으로 정렬합니다.
sorted_data = sorted(indexed_data, key=lambda x: x[1])
print(sorted_data)
'''

'\ndata = [-15, 1, -13, 3, -11, 5, -9, 7, -7, 9, -5, 11, -3, 13, -1, 15]\n# 원소에 인덱스를 붙입니다.\nindexed_data = list(enumerate(data))\n# 값을 기준으로 정렬합니다.\nsorted_data = sorted(indexed_data, key=lambda x: x[1])\nprint(sorted_data)\n'

In [113]:
','.join([str((i,j)) for i,j in zip(npa_omega, change_vertical_position)])

'(-15, 0),(1, 2),(-13, 4),(3, 6),(-11, 8),(5, 10),(-9, 12),(7, 14),(-7, 1),(9, 3),(-5, 5),(11, 7),(-3, 9),(13, 11),(-1, 13),(15, 15)'

In [114]:
#tpcap=data_pcap[0]
sel_data_pcap = data_pcap[0:max_pcap_counter]
frame_cnt = 0
pre_azimuth = 0
frame_angle_arr = []
frame_image_arr = []
#pr = update_print()
#pbar = tqdm(total=len(sel_data_pcap)*12, bar_format="{l_bar}{bar}|{n_fmt}/{total_fmt} [{postfix}]")
#pbar = tqdm(total=len(sel_data_pcap)*12, bar_format="{l_bar}{bar}|{n_fmt}/{total_fmt}: {postfix['message']}")
#pbar = tqdm(total=len(sel_data_pcap)*12)
#pbar.bar_format = "{l_bar}{bar}|{n_fmt}/{total_fmt}: {postfix[message]}"
#pbar = tqdm(total=len(sel_data_pcap)*12, bar_format="{l_bar}{bar}|{n_fmt}/{total_fmt} [{postfix}]")
#pbar = tqdm(total=len(sel_data_pcap)*12, bar_format="{l_bar}{bar}|{n_fmt}/{total_fmt}{postfix}")
pbar = tqdm(total=len(sel_data_pcap)*12, bar_format="{l_bar}{bar}|{n_fmt}/{total_fmt}{postfix}")

for t, tpcap in enumerate(sel_data_pcap):
    data = np.frombuffer(tpcap, dtype=np.uint8).astype(np.uint32)
    
    # frame length = 1206
    timestamp = read_uint32(data, 1200)
    factory = data[1204:]

    # veldyne has 12 blocks each 100 bytes data
    # data-legth = 1206 bytes
    blocks = data[0:1200].reshape(12, 100) 
    list_azimuth = [np.dot(blocks[i][2:4], [1, 256]) for i,v in enumerate(blocks)]
    #print(f'12 azimuth={list_azimuth}')

    distances = []
    intensities = []
    azimuth_per_block = []
    # iteratie through each block
    for i, blk in enumerate(blocks):
        dists, intens, angles = read_firing_data(blk)
        #print(f'len(dists)={len(dists)},angles={angles}, dists={dists}')
        #print(f'{int(angles*5), int(angles*5)+1}')
        distances.append(dists)
        intensities.append(intens)
        azimuth_per_block.append(angles)
        #print(f'i,list_azimuth[i]={i,list_azimuth[i]}')
        if pre_azimuth < list_azimuth[i]:
            pre_azimuth = list_azimuth[i]
            frame_angle_arr = attach_2d_list(frame_angle_arr, [list_azimuth[i]/100], False)
            frame_image_arr = attach_range_image_list(frame_image_arr, dists, False)
        else:
            pre_azimuth = 0 
            frame_cnt += 1
            #print(f'\rframe changed at {i:3d}th datablock in {t:4d}th UPD packet')
            #pr.print(f'frame changed at {i:3d}th datablock in {t:4d}th UPD packet')
            #pbar.set_postfix({"message": f'frame changed at {i:3d}th datablock in {t:4d}th UPD packet'}, refresh=False)
            #pbar.set_postfix({"message": f'frame changed at {i:3d}th datablock in {t:4d}th UPD packet'}, refresh=True)
            #pbar.set_postfix({"frame changed at ": f'{i:3d}th datablock in {t:4d}th UPD packet'}, refresh=True)
            #pbar.set_postfix({"": f'frame changed at {i:3d}th datablock in {t:4d}th UPD packet'}, refresh=True)
            pbar.set_postfix({"message": f"{frame_cnt:3d}th frame start at {i:3d}th datablock in {t:4d}th UPD packet"}, refresh=True)
            frame_angle_arr = attach_2d_list(frame_angle_arr, [list_azimuth[i]/100], True)
            frame_image_arr = attach_range_image_list(frame_image_arr, dists, True)
        pbar.update(1)
        #print(f'frame_angle_arr={frame_angle_arr}')
                                  
    #print(f'frame_angle_arr={frame_image_arr}')        
    #print(f'len(blocks)={len(blocks)}')
    #print(f'azimuth_per_block={azimuth_per_block}')

    azimuth_per_block = np.array(azimuth_per_block)
    #print(f'azimuth_per_block={azimuth_per_block}')

    ## Note: all these arrray have th same size, number of firing in one packet
    azimuth = calc_precise_azimuth_2(azimuth_per_block).reshape(12, 32)
    distances = np.array(distances)
    intensities = np.array(intensities)
    #print(f'len(distances)={len(dists)}, len(azimuth)={len(azimuth)}')

    # now calculate the cartesian coordinate of each point
    X, Y, Z = calc_cart_coord(distances, azimuth)
    #print(f'len(X)={len(X)}, len(Z)={len(Z)} / len(X[0])={len(X[0])}, len(Z[0])={len(Z[0])}')
    #print(f'X={X}')
    #print(f'Y={Y}')

    # calculating timestamp [microsec] of each firing
    timestamps = timestamp + npa_timing_offsets

    X = X.flatten()
    Y = Y.flatten()
    Z = Z.flatten()
    #print(f'len(X)={len(X)}, len(Z)={len(Z)}')
    
    intensities = intensities.flatten()
    azimuth = azimuth.flatten()
    timestamps = timestamps.flatten()
    distances = distances.flatten() * FACTOR_MM2CM * FACTOR_CM2M
    

    # remeber the last azimuth for roll over checking
    cur_azimuth = azimuth
    #print(f'frame_angle_arr={frame_image_arr}')
    


    # print(X, Y, Z, intensities, azimuth, timestamps, distances)

  0%|          |0/1831308

In [115]:
import plotly.graph_objs as go
import plotly

heatmap = go.Heatmap(z=frame_image_arr[1])

# 그래프 설정
layout = go.Layout(title="2D Array Heatmap")
fig = go.Figure(data=[heatmap], layout=layout)

# 그래프 출력
plotly.io.show(fig)

In [116]:
import plotly.graph_objs as go
import plotly.subplots as sp
import ipywidgets as widgets
from IPython.display import display

def plot_heatmap(data):
    heatmap = go.Heatmap(z=data)
    layout = go.Layout(title="2D Array Heatmap")
    fig = go.Figure(data=[heatmap], layout=layout)
    return fig

# 슬라이더를 사용하여 시간 단계를 변경할 수 있는 그래프 생성
@widgets.interact(time_step=(0, len(frame_image_arr) - 1))
def update(time_step=0):
    return plotly.io.show(plot_heatmap(frame_image_arr[time_step]))

interactive(children=(IntSlider(value=0, description='time_step', max=2025), Output()), _dom_classes=('widget-…

In [30]:
import plotly.graph_objs as go
import ipywidgets as widgets
import plotly.io as pio
from IPython.display import display

def plot_heatmap(data):
    heatmap = go.Heatmap(z=data)
    layout = go.Layout(title="2D Array Heatmap")
    fig = go.Figure(data=[heatmap], layout=layout)
    return fig

def update(time_step=0):
    fig = plot_heatmap(frame_image_arr[time_step])
    pio.show(fig)

# 슬라이더 생성
time_step_slider = widgets.IntSlider(min=0, max=len(frame_image_arr) - 1, step=1, description='Time step:')

# Play 위젯 생성
play_widget = widgets.Play(
    value=0,
    min=0,
    max=len(frame_image_arr) - 1,
    step=1,
    interval=100,  # 500ms 간격으로 값이 변경됩니다.
    description="Press play"
)

# Play 위젯과 슬라이더를 연결
widgets.jslink((play_widget, 'value'), (time_step_slider, 'value'))

# 슬라이더 값이 변경될 때마다 update 함수를 호출
interactive_output = widgets.interactive_output(update, {'time_step': time_step_slider})

# 위젯을 표시
display(widgets.HBox([play_widget, time_step_slider]))
display(interactive_output)

Output()

In [31]:
print(max_pcap_counter)
#frame_image_arr[time_step]

182036


In [105]:
print(len(frame_image_arr))
pcap_frame = np.array(frame_image_arr)
#pcap_frame=pcap_frame.reshape(16,len(frame_image_arr))
#pcap_frame = pcap_frame.reshape(16,2026)
print(pcap_frame.shape)

if len(pcap_frame[1][15]) >=1800:
    print(pcap_frame[20][15])
    
#for i in range(2):
#    path = "./"+"rimg_" + str(i)+".rimg"
#    print(path)
#    write_rimg(path, frame_image_arr[i])

2026
(2026, 16)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

c:\Temp\ipykernel_30460\1076844846.py:2: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [81]:
print(pcap_frame[2025][15].transpose())

AttributeError: 'list' object has no attribute 'transpose'